In [1]:
!pip list

Package                       Version
----------------------------- -------------------
absl-py                       0.12.0
alabaster                     0.7.12
albumentations                0.1.12
altair                        4.1.0
appdirs                       1.4.4
argcomplete                   1.12.3
argon2-cffi                   20.1.0
arviz                         0.11.2
astor                         0.8.1
astropy                       4.3.1
astunparse                    1.6.3
async-generator               1.10
atari-py                      0.2.9
atomicwrites                  1.4.0
attrs                         21.2.0
audioread                     2.1.9
autograd                      1.3
Babel                         2.9.1
backcall                      0.2.0
beautifulsoup4                4.6.3
bleach                        4.0.0
blis                          0.4.1
bokeh                         2.3.3
Bottleneck                    1.3.2
branca                        0.4.2
bs4     

In [2]:
pip install happytransformer

     |████████████████████████████████| 44 kB 1.4 MB/s 
     |████████████████████████████████| 264 kB 9.0 MB/s 
     |████████████████████████████████| 1.2 MB 48.4 MB/s 
     |████████████████████████████████| 2.6 MB 52.0 MB/s 
     |████████████████████████████████| 243 kB 38.6 MB/s 
     |████████████████████████████████| 43 kB 2.0 MB/s 
     |████████████████████████████████| 118 kB 51.6 MB/s 
     |████████████████████████████████| 895 kB 27.3 MB/s 
     |████████████████████████████████| 636 kB 60.5 MB/s 
     |████████████████████████████████| 3.3 MB 48.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
!nvidia-smi

Mon Aug 16 07:11:15 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
from happytransformer import HappyQuestionAnswering, QATrainArgs, QATestArgs, QAEvalArgs
import torch
import pandas as pd
import shutil
from google.colab import drive
from psutil import virtual_memory


In [5]:
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [6]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Mon Aug 16 07:11:34 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
roberta_qa = HappyQuestionAnswering("ROBERTA","deepset/roberta-large-squad2")


Downloading:   0%|          | 0.00/696 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

08/16/2021 07:12:50 - INFO - happytransformer.happy_transformer -   Using model: cuda


In [9]:
torch.version.cuda

'10.2'

In [10]:
roberta_qa.model

RobertaForQuestionAnswering(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              

In [11]:
roberta_qa.model_name

'deepset/roberta-large-squad2'

In [12]:
roberta_qa.model_type


'ROBERTA'

In [13]:
roberta_train_args = QATrainArgs(learning_rate=2e-5, batch_size=1)

In [14]:
roberta_train_args

QATrainArgs(learning_rate=2e-05, num_train_epochs=3.0, weight_decay=0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, save_preprocessed_data=False, save_preprocessed_data_path='', load_preprocessed_data=False, load_preprocessed_data_path='', batch_size=1)

In [15]:
pd.read_csv("./drive/MyDrive/Research_Transformer/train/train_qa_10k.csv")

,context,question,answer_text,answer_start
0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,in the late 1990s,269
1,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,singing and dancing,207
2,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,2003,526
3,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?,"Houston, Texas",166
4,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?,late 1990s,276
...,...,...,...,...
9995,The Oklahoman is Oklahoma City's major daily n...,What is the name of The Oklahoman's website?,NewsOK.com,103
9996,The Oklahoman is Oklahoma City's major daily n...,What is Oklahoma Cities newsweekly?,Oklahoma Gazette,150
9997,The Oklahoman is Oklahoma City's major daily n...,What is Oklahoma Cities daily business newspaper?,The Journal Record,338
9998,There are numerous community and international...,Which side is the Black Chronicles Headquarters?,Eastside,156


In [31]:
!nvidia-smi

Mon Aug 16 11:51:10 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P0    37W / 250W |   8617MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [17]:
train_output = roberta_qa.train("./drive/MyDrive/Research_Transformer/train/train_qa_10k.csv",args=roberta_train_args)


08/16/2021 07:13:38 - INFO - happytransformer.happy_transformer -   Preprocessing dataset...
***** Running training *****
  Num examples = 10000
  Num Epochs = 3
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 30000


Step,Training Loss
500,1.147300
1000,1.506900
1500,1.578000
2000,1.427700
2500,1.408700
3000,1.584900
3500,1.427600
4000,1.535000
4500,1.426000
5000,1.334500


Saving model checkpoint to /tmp/tmpc01xd_v6/checkpoint-500
Configuration saved in /tmp/tmpc01xd_v6/checkpoint-500/config.json
Model weights saved in /tmp/tmpc01xd_v6/checkpoint-500/pytorch_model.bin
tokenizer config file saved in /tmp/tmpc01xd_v6/checkpoint-500/tokenizer_config.json
Special tokens file saved in /tmp/tmpc01xd_v6/checkpoint-500/special_tokens_map.json
Saving model checkpoint to /tmp/tmpc01xd_v6/checkpoint-1000
Configuration saved in /tmp/tmpc01xd_v6/checkpoint-1000/config.json
Model weights saved in /tmp/tmpc01xd_v6/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in /tmp/tmpc01xd_v6/checkpoint-1000/tokenizer_config.json
Special tokens file saved in /tmp/tmpc01xd_v6/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to /tmp/tmpc01xd_v6/checkpoint-1500
Configuration saved in /tmp/tmpc01xd_v6/checkpoint-1500/config.json
Model weights saved in /tmp/tmpc01xd_v6/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in /tmp/tmpc01xd_v6/checkpo

In [33]:
pd.read_csv('./drive/MyDrive/Research_Transformer/val/val_qa_6k.csv')

,context,question,answer_text,answer_start
0,Another possible cause of diarrhea is irritabl...,What is IBS?,irritable bowel syndrome,38
1,Another possible cause of diarrhea is irritabl...,What symptoms do you have with IBS?,abdominal discomfort relieved by defecation an...,98
2,Another possible cause of diarrhea is irritabl...,How can symptoms be managed with IBS?,"a combination of dietary changes, soluble fibe...",302
3,Poverty is a good indicator of the rate of inf...,What is a good indicator of the rate of infect...,Poverty,0
4,Poverty is a good indicator of the rate of inf...,Why does living in poverty raise your risk of ...,from the conditions under which impoverished p...,143
...,...,...,...,...
5995,"BJTs have three terminals, corresponding to th...",How can one find the amount of collector current?,β (common-emitter current gain) times the base...,945
5996,"BJTs have three terminals, corresponding to th...",What is the usual collector current for small-...,greater than 100,1017
5997,"In a FET, the drain-to-source current flows vi...",What determines the conductivity in a FET?,electric field that is produced when a voltage...,156
5998,"In a FET, the drain-to-source current flows vi...",What controls the current between the drain an...,the voltage applied between the gate and source,325


In [34]:
roberta_eval_args = QAEvalArgs()

In [35]:
roberta_eval_args

QAEvalArgs(batch_size=1, save_preprocessed_data=False, save_preprocessed_data_path='', load_preprocessed_data=False, load_preprocessed_data_path='')

In [38]:
eval_output = roberta_qa.eval("./drive/MyDrive/Research_Transformer/val/val_qa_6k.csv", args=roberta_eval_args)


PyTorch: setting up devices
***** Running Evaluation *****
  Num examples = 6000
  Batch size = 1


In [40]:
eval_output


EvalResult(loss=2.418659210205078)

In [41]:
!mkdir ./drive/MyDrive/Research_Transformer/models

mkdir: cannot create directory ‘./drive/MyDrive/Research_Transformer/models’: File exists


In [42]:
!mkdir ./drive/MyDrive/Research_Transformer/models/roberta

In [43]:
!mkdir ./drive/MyDrive/Research_Transformer/models/roberta/model

In [44]:
!ls ./drive/MyDrive/Research_Transformer/

models	squad  test  train  val


In [45]:
!ls ./drive/MyDrive/Research_Transformer/models/

albert	roberta


In [46]:
!ls ./drive/MyDrive/Research_Transformer/models/roberta/

model


In [47]:
roberta_eval_loss = {'eval_loss' : [eval_output.loss]}


In [49]:
roberta_eval_loss


{'eval_loss': [2.418659210205078]}

In [51]:
roberta_eval_loss_df = pd.DataFrame(roberta_eval_loss)


In [54]:
roberta_eval_loss_df


,eval_loss
0,2.418659


In [56]:
roberta_eval_loss_df.to_csv(r'./drive/MyDrive/Research_Transformer/models/roberta/roberta_eval_loss.csv', index=False)


In [58]:
pd.read_csv('./drive/MyDrive/Research_Transformer/models/roberta/roberta_eval_loss.csv')


,eval_loss
0,2.418659


In [60]:
roberta_sample_result = roberta_qa.answer_question("Macedonia was an ancient kingdom on the periphery of Archaic and Classical Greece, and later the dominant state of Hellenistic Greece. The kingdom was founded and initially ruled by the Argead dynasty, followed by the Antipatrid and Antigonid dynasties. Home to the ancient Macedonians, it originated on the northeastern part of the Greek peninsula. Before the 4th century BC, it was a small kingdom outside of the area dominated by the city-states of Athens, Sparta and Thebes, and briefly subordinate to Achaemenid Persia.",
                         "Who ruled Macedonia?")


In [62]:
roberta_sample_result


[QuestionAnsweringResult(answer='the Argead dynasty', score=0.9980077743530273, start=182, end=200)]

In [63]:
pd.read_csv('./drive/MyDrive/Research_Transformer/test/test_qa_5k.csv')


,context,question,answer_text,answer_start
0,Required attendance at school is 10 years for ...,How long are females required to go to school?,11 years,56
1,Required attendance at school is 10 years for ...,What was the literacy rate on Tuvalu in 2002?,99.0%,112
2,Required attendance at school is 10 years for ...,How many students were in Tuvalu schools in 2010?,"1,918",146
3,Required attendance at school is 10 years for ...,What si the teacher-student ratio for Tuvalu s...,1:18,292
4,Community Training Centres (CTCs) have been es...,What type of school has Tuvalu set up on each ...,Community Training Centres,0
...,...,...,...,...
4995,"In the years that followed, Eisenhower increas...",How many soldiers did Eisenhower ultimately se...,900,106
4996,"In the years that followed, Eisenhower increas...",How many days did the President of South Vietn...,ten,313
4997,"In the years that followed, Eisenhower increas...",Where was a parade held for the President of S...,New York City,416
4998,"In the years that followed, Eisenhower increas...",Who was Secretary of State in 1957?,John Foster Dulles,497


In [64]:
pd.read_csv('./drive/MyDrive/Research_Transformer/test/test_qa_without_answer_5k.csv')


,context,question
0,Required attendance at school is 10 years for ...,How long are females required to go to school?
1,Required attendance at school is 10 years for ...,What was the literacy rate on Tuvalu in 2002?
2,Required attendance at school is 10 years for ...,How many students were in Tuvalu schools in 2010?
3,Required attendance at school is 10 years for ...,What si the teacher-student ratio for Tuvalu s...
4,Community Training Centres (CTCs) have been es...,What type of school has Tuvalu set up on each ...
...,...,...
4995,"In the years that followed, Eisenhower increas...",How many soldiers did Eisenhower ultimately se...
4996,"In the years that followed, Eisenhower increas...",How many days did the President of South Vietn...
4997,"In the years that followed, Eisenhower increas...",Where was a parade held for the President of S...
4998,"In the years that followed, Eisenhower increas...",Who was Secretary of State in 1957?


In [65]:
roberta_test_args = QATestArgs()


In [67]:
roberta_test_args


QATestArgs(save_preprocessed_data=False, save_preprocessed_data_path='', load_preprocessed_data=False, load_preprocessed_data_path='')

In [70]:
roberta_result = roberta_qa.test('./drive/MyDrive/Research_Transformer/test/test_qa_without_answer_5k.csv', args=roberta_test_args)


5000it [02:42, 30.81it/s]


In [72]:
roberta_result


[QuestionAnsweringResult(answer='11 years', score=0.9999619722366333, start=56, end=64),
 QuestionAnsweringResult(answer='99.0%', score=0.9999599456787109, start=112, end=117),
 QuestionAnsweringResult(answer='1,918', score=0.9999656677246094, start=146, end=151),
 QuestionAnsweringResult(answer='1:18', score=0.9496452808380127, start=292, end=296),
 QuestionAnsweringResult(answer='Community Training Centres', score=0.9997596740722656, start=0, end=26),
 QuestionAnsweringResult(answer='vocational training', score=0.8054276704788208, start=115, end=134),
 QuestionAnsweringResult(answer='secondary education', score=0.9999610185623169, start=231, end=250),
 QuestionAnsweringResult(answer='Adults', score=0.9999865889549255, start=497, end=503),
 QuestionAnsweringResult(answer='basic carpentry, gardening and farming, sewing and cooking', score=0.8252884149551392, start=279, end=337),
 QuestionAnsweringResult(answer='plants and trees', score=0.9681934714317322, start=41, end=57),
 QuestionAn

In [74]:
roberta_result[4997].answer

'New York City'

In [76]:
roberta_result_df = pd.DataFrame([vars(a) for a in roberta_result])


In [78]:
roberta_result_df


,answer,score,start,end
0,11 years,0.999962,56,64
1,99.0%,0.999960,112,117
2,"1,918",0.999966,146,151
3,1:18,0.949645,292,296
4,Community Training Centres,0.999760,0,26
...,...,...,...,...
4995,900,0.999896,106,109
4996,ten,0.993736,313,316
4997,New York City,0.995167,416,429
4998,John Foster Dulles,0.999938,497,515


In [79]:
roberta_result_df.to_csv(r'./drive/MyDrive/Research_Transformer/models/roberta/roberta_result.csv', index=False)


In [82]:
pd.read_csv('./drive/MyDrive/Research_Transformer/models/roberta/roberta_result.csv')


,answer,score,start,end
0,11 years,0.999962,56,64
1,99.0%,0.999960,112,117
2,"1,918",0.999966,146,151
3,1:18,0.949645,292,296
4,Community Training Centres,0.999760,0,26
...,...,...,...,...
4995,900,0.999896,106,109
4996,ten,0.993736,313,316
4997,New York City,0.995167,416,429
4998,John Foster Dulles,0.999938,497,515


In [83]:
pd.read_csv('./drive/MyDrive/Research_Transformer/test/test_qa_5k.csv')


,context,question,answer_text,answer_start
0,Required attendance at school is 10 years for ...,How long are females required to go to school?,11 years,56
1,Required attendance at school is 10 years for ...,What was the literacy rate on Tuvalu in 2002?,99.0%,112
2,Required attendance at school is 10 years for ...,How many students were in Tuvalu schools in 2010?,"1,918",146
3,Required attendance at school is 10 years for ...,What si the teacher-student ratio for Tuvalu s...,1:18,292
4,Community Training Centres (CTCs) have been es...,What type of school has Tuvalu set up on each ...,Community Training Centres,0
...,...,...,...,...
4995,"In the years that followed, Eisenhower increas...",How many soldiers did Eisenhower ultimately se...,900,106
4996,"In the years that followed, Eisenhower increas...",How many days did the President of South Vietn...,ten,313
4997,"In the years that followed, Eisenhower increas...",Where was a parade held for the President of S...,New York City,416
4998,"In the years that followed, Eisenhower increas...",Who was Secretary of State in 1957?,John Foster Dulles,497


In [84]:
roberta_qa.save('./drive/MyDrive/Research_Transformer/models/roberta/model')

Configuration saved in ./drive/MyDrive/Research_Transformer/models/roberta/model/config.json
Model weights saved in ./drive/MyDrive/Research_Transformer/models/roberta/model/pytorch_model.bin
tokenizer config file saved in ./drive/MyDrive/Research_Transformer/models/roberta/model/tokenizer_config.json
Special tokens file saved in ./drive/MyDrive/Research_Transformer/models/roberta/model/special_tokens_map.json


In [89]:
!ls -lhac './drive/MyDrive/Research_Transformer/models/roberta/model'


total 1.4G
-rw------- 1 root root  748 Aug 16 12:30 config.json
-rw------- 1 root root 446K Aug 16 12:30 merges.txt
-rw------- 1 root root 1.4G Aug 16 12:30 pytorch_model.bin
-rw------- 1 root root  239 Aug 16 12:30 special_tokens_map.json
-rw------- 1 root root 1.2K Aug 16 12:30 tokenizer_config.json
-rw------- 1 root root 1.3M Aug 16 12:30 tokenizer.json
-rw------- 1 root root 780K Aug 16 12:30 vocab.json
